In [1]:

import os


import featureform as ff
from dotenv import load_dotenv


load_dotenv()

debug = False
client = ff.Client(host="localhost:7878", insecure=True, debug=debug)
# client = ff.Client(host="internal-sandbox2.featureform.com")

In [2]:
redis = ff.register_redis(
    name="redis-quickswtart3",
    host="localhost",  # The internal dns name for redis
    port=6379,
    description="A Redis deployment we created for the Featureform quickstart",
)

client.apply()

Applying Run: 2025-01-18t17-47-00
Creating User default_owner 


Output()

Creating Provider redis-quickswtart3 



In [3]:
snow = ff.register_snowflake(
    name="hi332",
    username=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    organization=os.getenv("SNOWFLAKE_ORG"),
    database="DEMO",
    schema="ALI",
    warehouse="COMPUTE_WH",
    catalog=ff.SnowflakeCatalog(
        external_volume="iceberg_external_volume",
        base_location="impl_test",
        table_config=ff.SnowflakeDynamicTableConfig(
            # target_lag='1 hours',
            target_lag='DOWNSTREAM',
            refresh_mode=ff.RefreshMode.AUTO,
            initialize=ff.Initialize.ON_CREATE,
        ),
    )
)
client.apply()

Applying Run: 2025-01-18t17-47-03
Creating Provider hi332 


Output()

In [4]:
# type(snow)

In [5]:
# client.delete(snow)

In [6]:
# from featureform import OfflineProvider

# isinstance(snow, OfflineProvider)

In [7]:
# client.delete(snow)

In [8]:
trx = snow.register_table(name="tblll", table="TRANACTIONZ", database="DEMO", schema="ALI")

client.apply()

Applying Run: 2025-01-18t17-47-07
Creating Source Variant tblll 2025-01-18t17-47-07


Output()

In [9]:
# client.delete(trx)

In [10]:
# type(trx)

In [11]:
# client.delete(trx)

In [12]:
@snow.sql_transformation(inputs=[trx])
def label_transactionz2(tbl):
    return "SELECT customerid, isfraud, CAST(timestamp AS TIMESTAMP_NTZ(6)) AS timestamp FROM {{tbl}}"

client.apply()

Output()

Applying Run: 2025-01-18t17-47-13
Creating Source Variant label_transactionz2 2025-01-18t17-47-13



In [13]:
@snow.sql_transformation(inputs=[trx])
def avg_daily_user_transaction2(tbl):
    return "SELECT CustomerID AS user_id, avg(TransactionAmount) " \
           "AS avg_transaction_amt, CAST(timestamp AS TIMESTAMP_NTZ(6)) AS processed_on FROM {{tbl}} GROUP BY user_id, processed_on"

client.apply()

Output()

Applying Run: 2025-01-18t17-47-24
Creating Source Variant avg_daily_user_transaction2 2025-01-18t17-47-24



In [14]:
# client.delete(avg_daily_user_transaction3)

In [19]:
@ff.entity
class User:
    alif2 = ff.Feature(
        avg_daily_user_transaction2[["user_id", "avg_transaction_amt", "processed_on"]],
        type=ff.Float32,
        inference_store=redis,
    )
    # fraudulent = ff.Label(
    #     label_transactions[["customerid", "isfraud", "timestamp"]], type=ff.Bool
    # )
    fraudulent = ff.Label(
        label_transactionz2[["customerid", "isfraud", "timestamp"]], type=ff.Bool
    )

client.apply()

Output()

Applying Run: 2025-01-18t17-47-51
Creating Entity user 
Creating Feature Variant alif2 2025-01-18t17-47-51
Looks like an equivalent feature variant already exists, going to use its variant:  2025-01-18t17-47-32
Creating Label Variant fraudulent 2025-01-18t17-47-51



In [16]:
# type(label_transactionz)

In [17]:
# client.delete(User.fraudulent)

Deleting resource async


In [20]:
ts = ff.register_training_set("666555999", label=User.fraudulent, features=[User.alif2])

client.apply()

Applying Run: 2025-01-18t17-48-13
Creating Trainingset Variant 666555999 2025-01-18t17-48-13


Output()

In [ ]:
# client.prune(avg_daily_user_transaction2)

In [21]:
client.delete(ts)

Deleting resource async


In [ ]:
# client.delete(ts)

In [ ]:
@snow.sql_transformation(inputs=[trx])
def myfirsttfagain235(tbl):
    return """
    SELECT customerid, isfraud, CAST(timestamp AS TIMESTAMP_NTZ(6)) AS timestamp FROM {{tbl}}
    """
client.apply()

In [ ]:
@snow.sql_transformation(inputs=[myfirsttfagain24])
def alitfsnow5(tbl):
    return """
    SELECT customerid, isfraud, CAST(timestamp AS TIMESTAMP_NTZ(6)) AS timestamp FROM {{tbl}}
    """
client.apply()

In [ ]:
from featureform import ResourceType

client.delete('myfirsttfagain24', '2024-12-19t16-45-59', ResourceType.TRANSFORMATION)

In [ ]:
@ff.entity
class User:
    f_trx = ff.Feature(
        trx[
            ["CUSTOMERID", "CUSTACCOUNTBALANCE", "TIMESTAMP"]
        ],  # We can optional include the `timestamp_column` "timestamp" here
        type=ff.Float32,
        inference_store=redis,
    )

client.apply()